In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import osmnx as ox
from descartes import PolygonPatch
from shapely.geometry import Point, LineString, Polygon
ox.config(log_console=True, use_cache=True)
ox.__version__

In [ ]:
network_type = 'walk'
trip_time = 30 #in minutes
trip_length = 1000 #in meters
travel_speed = 4.5 #walking speed in km/hour
lat = YOUR_LONGITUDE
lon = YOUR_LATITUDE
iso_color = '#4edce6'

In [ ]:
G = ox.graph_from_point((lat, lon), distance=1000, network_type=network_type)
ox.plot_graph(G, fig_height=12)

In [ ]:
# find the centermost node and then project the graph to UTM
center = ox.get_nearest_node(G, (lat, lon), method='euclidean', return_dist=True)
G1 = ox.project_graph(G)
center_node = center[0]

In [ ]:
# add an edge attribute for time in minutes required to traverse each edge
meters_per_minute = travel_speed * 1000 / 60 #km per hour to m per minute
for u, v, k, data in G1.edges(data=True, keys=True):
    data['time'] = data['length'] / meters_per_minute

In [ ]:
# color the nodes according to isochrone then plot the street network
def create_subgraph(G, center_node, radius, distance):
    subgraph = nx.ego_graph(G, center_node, radius=radius, distance=distance)
    node_colors = {}
    for node in subgraph.nodes():
        node_colors[node] = iso_color
    nc = [node_colors[node] if node in node_colors else 'none' for node in G.nodes()]
    ns = [20 if node in node_colors else 0 for node in G.nodes()]
    fig, ax = ox.plot_graph(G, fig_height=12, node_color=nc, node_size=ns, node_zorder=2)

In [ ]:
create_subgraph(G1, center_node, trip_length, 'length')

In [ ]:
def make_iso_polys(G, center_node, radius, distance, edge_buff=25, node_buff=50, infill=False):
    isochrone_polys = []
    subgraph = nx.ego_graph(G, center_node, radius=radius, distance=distance)

    node_points = [Point((data['x'], data['y'])) for node, data in subgraph.nodes(data=True)]
    nodes_gdf = gpd.GeoDataFrame({'id': subgraph.nodes()}, geometry=node_points)
    nodes_gdf = nodes_gdf.set_index('id')

    edge_lines = []
    for n_fr, n_to in subgraph.edges():
        f = nodes_gdf.loc[n_fr].geometry
        t = nodes_gdf.loc[n_to].geometry
        edge_lines.append(LineString([f,t]))

    n = nodes_gdf.buffer(node_buff).geometry
    e = gpd.GeoSeries(edge_lines).buffer(edge_buff).geometry
    all_gs = list(n) + list(e)
    new_iso = gpd.GeoSeries(all_gs).unary_union
        
    # try to fill in surrounded areas so shapes will appear solid and blocks without white space inside them
    if infill:
        new_iso = Polygon(new_iso.exterior)
    return new_iso

In [ ]:
iso_length = make_iso_polys(G1, center_node, trip_length, 'length', edge_buff=25, node_buff=0, infill=True)
fig, ax = ox.plot_graph(G1, fig_height=12, show=False, close=False, edge_color='k', edge_alpha=0.2, node_color='none')

patch = PolygonPatch(iso_length, fc=iso_color, ec='none', alpha=0.6, zorder=-1)
ax.add_patch(patch)
plt.show()

In [ ]:
from geopandas import GeoSeries, GeoDataFrame
gdf = GeoDataFrame(geometry=GeoSeries(iso_length))

In [ ]:
gdf.crs = G1.graph['crs']
gdf = gdf.to_crs("EPSG:4326")
bounds = gdf.bounds

In [ ]:
gdf.head()

In [ ]:
import os;
import time;

from cartoframes import to_carto
from cartoframes.auth import Credentials, set_default_credentials
from cartoframes.viz import Map, Layer

CARTO_BASE_URL = os.environ['CARTO_API_URL']
CARTO_API_KEY = os.environ['CARTO_API_KEY']
CARTO_USER_NAME = os.environ['CARTO_USER_NAME']

In [ ]:
Map(Layer(gdf))

In [ ]:
creds = Credentials(username=CARTO_USER_NAME, api_key=CARTO_API_KEY)
set_default_credentials(creds)
dataset_name = 'onekmiso'

In [ ]:
to_carto(gdf, dataset_name, if_exists='replace', log_enabled=True)

In [ ]:
# Carto Python SDK
from carto.auth import APIKeyAuthClient
from carto.maps import NamedMapManager

# carto-print
from carto.print import Printer

In [ ]:
def create_named_map(auth_client, username, dataset_name, map_name, factor):
    template = {
      "version": "0.0.1",
      "name": map_name,
      "auth": {
        "method": "open"
      },
      "placeholders": {},
      "view": {},
      "layergroup": {
        "version": "1.0.1",
        "layers": [
          {
            "type": "http",
            "options": {
                "urlTemplate": "http://a.tile.stamen.com/toner/{z}/{x}/{y}.png"
                #"urlTemplate": "https://a.basemaps.cartocdn.com/rastertiles/voyager_labels_under/{z}/{x}/{y}.png"
            }
          },
          {
            "type": "cartodb",
            "options": {
              "cartocss_version": "2.1.1",
              "cartocss": '''#layer {
                              polygon-fill: #2a2a2a;
                              polygon-opacity: 0;
                            }
                            #layer::outline {
                              line-width: 6 * %d;
                              line-color: #4edce6;
                              line-opacity: 1;
                              line-dasharray: 10, 3, 2, 3;
                            }''' % (factor),
              "sql": '''SELECT 1 AS cartodb_id,
                               the_geom,
                               the_geom_webmercator
                        FROM {dataset}'''.format(dataset=dataset_name)
            }
          },
          {
            "type": "cartodb",
            "options": {
              "cartocss_version": "2.1.1",
              "cartocss": '''#layer {
                              marker-width: 40 * %d;
                              marker-fill: #EE4D5A;
                              marker-fill-opacity: 0.9;
                              marker-file: url('https://s3.amazonaws.com/com.cartodb.users-assets.production/maki-icons/cross-18.svg');
                              marker-allow-overlap: true;
                            }
                            ''' % (factor),
              "sql": '''SELECT 1 AS cartodb_id,
                               ST_SetSRID( ST_Point( {lon}, {lat}), 4326) as the_geom,
                               ST_Transform(ST_SetSRID( ST_Point( {lon}, {lat}), 4326), 3857) as the_geom_webmercator
                     '''.format(lon=lon, lat=lat)
            }
          }
        ]
      }
    }

    named_map_manager = NamedMapManager(auth_client)

    try:
      named_map = named_map_manager.get(map_name)
      if named_map is not None:
          named_map.client = auth_client
          named_map.delete()
    except Exception as e:
      #ignore
      print(e)

    return named_map_manager.create(template=template)

In [ ]:
CARTO_BASE_URL

In [ ]:
auth_client = APIKeyAuthClient(CARTO_BASE_URL, CARTO_API_KEY)
DPI = 72
FACTOR = DPI / 72.0
map_name = 'tpl_' + dataset_name + str(int(round(time.time() * 1000)))
create_named_map(auth_client, CARTO_USER_NAME, dataset_name, map_name, FACTOR)

In [ ]:
map = {
  'username': CARTO_USER_NAME,
  'map_id': map_name,
  'width': 29.7,
  'height': 21,
  'dpi': DPI,
  'zoom': 1,
  'bounds': f"{bounds['minx'][0]},{bounds['miny'][0]},{bounds['maxx'][0]},{bounds['maxy'][0]}",
  'api_key': CARTO_API_KEY
}

p = Printer(map['username'], map['map_id'], map['api_key'], map['width'], map['height'], map['zoom'], map['bounds'], map['dpi'], 'RGBA')
image_path = p.export('.')

In [ ]:
from IPython.display import Image
Image(filename=image_path)

In [ ]:
named_map_manager = NamedMapManager(auth_client)

try:
  named_map = named_map_manager.get(map_name)
  if named_map is not None:
      named_map.client = auth_client
      named_map.delete()
except Exception as e:
  #ignore
  print(e)